## Use NLP to discover the current product trends

### Topic modeling visualization

In [1]:
%pylab inline

import pandas as pd
import numpy as np
import pickle as pk
from scipy import sparse as sp
import re

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv('all.csv')
df1 = df[df.website =='ASOS']
docs = df1['alltext']
df.sample(5)

Unnamed: 0                                            alltext brand  \
5362        5362  Mamalicious tank top It’s got you covered all ...   NaN   
6345        6345  Missguided zig a zig ah slogan t-shirt in blac...   NaN   
6754        6754  Y.A.S Ruffle Detail Cotton Blouse Not for the ...   NaN   
7505        7505  Nobody's Child Gingham Cami Dress With Waist B...   NaN   
97            97  One-shoulder Top One-shoulder top in cotton je...    HM   

     category                                        description img_2  \
5362      Top  It’s got you covered all the way from bump to ...   NaN   
6345      Top  Crew neck ,Slogan print to chest ,We all know ...   NaN   
6754      Top  Not for the clumsy ones,Crew neck,Ruffle trims...   NaN   
7505    Dress  Everything looks better with a little bit of g...   NaN   
97        Top  One-shoulder top in cotton jersey with elastic...   NaN   

                   img_id                                            img_url  \
5362                  NaN  https://images.asos-media.com/products/mamalic...   
6345                  NaN  https://images.asos-media.com/products/missgui...   
6754                  NaN  https://images.asos-media.com/products/yas-ruf...   
7505                  NaN  https://images.asos-media.com/products/nobodys...   
97    97 One-shoulder Top  https://lp2.hm.com/hmgoepprod?set=source[/ad/3...   

                                               material   price  \
5362  Soft-touch stretch jersey,Why complicate thing...  $23.00   
6345  Basic jersey,Why complicate things?,Main: 100%...  $24.00   
6754  Lightweight woven fabric,The kind that doesn't...  $60.00   
7505  Woven fabric,The kind that doesn't stretch,Bel...  $42.00   
97                                          Cotton 100%  $14.99   

                                           product_name website    id  
5362                               Mamalicious tank top    ASOS  5362  
6345    Missguided zig a zig ah slogan t-shirt in black    ASOS  6345  
6754                  Y.A.S Ruffle Detail Cotton Blouse    ASOS  6754  
7505  Nobody's Child Gingham Cami Dress With Waist Belt    ASOS  7505  
97                                     One-shoulder Top      HM    97

In [3]:
print(docs.index)
docs.head()

Int64Index([ 3136,  3137,  3138,  3139,  3140,  3141,  3142,  3143,  3144,
             3145,
            ...
            10205, 10206, 10207, 10208, 10209, 10210, 10211, 10212, 10213,
            10214],
           dtype='int64', length=7079)


3136    ASOS DESIGN blouse with frill shoulder Lightwe...
3137    A Star Is Born Going Out Festival High Neck Bo...
3138    Converse Cons Skate Boarding Long Sleeve T-Shi...
3139    Vans White Finish Line Heritage Long Sleeve T-...
3140    ASOS DESIGN x glaad& Curve t-shirt with high n...
Name: alltext, dtype: object

### Pre-process and vectorize the documents

In [4]:
from nltk.tokenize import RegexpTokenizer

def preprocessDoc(docs):
    docs = np.array(docs)  # no longer have index
    tokenizer = RegexpTokenizer(r'\w+')
    for idx in range(len(docs)):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        
        stop = ['ASOS', 'macy', 'bloomingdale', "fashion nova", "YAS", "Ditsy", "Noisy", "May", "Ted","Baker", "River","Island", "Karen","Scott","PrettyLittleThing","Roxy","DESIGN","Chi", \
               "Alfani","Boohoo","Sofie","Schnoor","Ellesse", "Jeannie","TFNC","Sacred", "Hawk","Urban","Bliss","Puma","adidas", "Stella", \
               'cm', 'size', 'web id', 'approx', 'model', 'height', 'is', 'and', 'she', 'wearing', 'small', 'approximate', 'measurements', 'height', 
               'bust', 'waist', 'hips', 'made', 'usa', 'things', 'regular', 'right', 'kind', 'that', 'things']

        lst = r'|'.join([x.lower() for x in stop])

        docs[idx] = re.sub(lst,'',docs[idx])
        
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    docs = [[token for token in doc if not token.isdigit()] for doc in docs]  #remove numbers
    docs = [[token for token in doc if len(token) > 3] for doc in docs] #remove words that are short 
  
    return docs

docs = preprocessDoc(docs)

#### Compute bigrams/trigrams and remove less words and very common words:

In [5]:
from gensim.corpora import Dictionary
from gensim.models import Phrases
bigram = Phrases(docs, min_count=10, threshold=10)  # only ones that appear 10 times or more.
trigram = Phrases(bigram[docs])

for i in range(len(docs)):
    for token in bigram[docs[i]]:
        if '_' in token:
            docs[i].append(token)
#            print ("bi:", token)
    for token in trigram[docs[i]]:
        if token.count('_') > 2 :
            docs[i].append(token)
#            print ("tri:", token)

dictionary_ = Dictionary(docs)
print (len(dictionary_))
dictionary_.filter_extremes(no_below=10, no_above=0.2)
print (len(dictionary_))

/Users/kellyho/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


3133
1518


### Vectorize data

- bag-of-words : frequency of words

In [6]:
corpus = [dictionary_.doc2bow(doc) for doc in docs]
print('unique tokens: %d' % len(dictionary_)) #gensim.corpora.dictionary.Dictionary
print('Number of records: %d' % len(corpus)) #List

unique tokens: 1518
Number of records: 7079


## Train LDA model

In [7]:
from gensim.models import LdaModel

# Set training parameters.
num_topics = 6
chunksize = 500 #important #number of documents to be used in each training chunk. (how many documents to load into memory)
passes = 20 #important #number of passes through the corpus during training
iterations = 400 #maximum number of iterations through the corpus when inferring the topic distribution of a corpus
eval_every = 1  #log perplexity is estimated every that many updates. Setting this to one slows down training by ~2x.

# Make a index to word dictionary.
dictionary_[0]  #this is only to "load" the dictionary. Otherwise, you will get the value error. 
id2word = dictionary_.id2token

model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, alpha='auto', eta='auto', iterations=iterations, num_topics=num_topics, passes=passes, eval_every=eval_every)


In [8]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [9]:
pyLDAvis.gensim.prepare(model, corpus, dictionary_)

/Users/kellyho/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.191485 -0.105591       1        1  28.268829
5     -0.068228 -0.088658       2        1  24.257898
3      0.137705 -0.236720       3        1  17.915237
1      0.008620  0.227095       4        1  11.496816
4      0.122007  0.242691       5        1  10.247695
0     -0.391588 -0.038817       6        1   7.813528, topic_info=     Category         Freq                              Term        Total  \
term                                                                        
1135  Default  1018.000000                              maxi  1018.000000   
1     Default  1551.000000                            button  1551.000000   
12    Default  1272.000000                             close  1272.000000   
579   Default  1059.000000                              mini  1059.000000   
320   Default  1340.000000                            floral  1340.000000   
140   Default  1473.000000                            lining  1473.000000   
1199  Default   631.000000                             fully   631.000000   
1200  Default   631.000000                       fully_lined   631.000000   
19    Default  1451.000000                         fastening  1451.000000   
48    Default  1247.000000                              crew  1247.000000   
228   Default  1120.000000                              lace  1120.000000   
257   Default   604.000000                         conscious   604.000000   
136   Default   589.000000                             holds   589.000000   
210   Default   900.000000                              more   900.000000   
625   Default   568.000000                             skirt   568.000000   
5     Default   712.000000                           placket   712.000000   
10    Default   566.000000                           bodycon   566.000000   
258   Default   508.000000                   conscious_holds   508.000000   
2     Default   651.000000                    button_placket   651.000000   
375   Default   485.000000                              slim   485.000000   
248   Default   786.000000                            ruffle   786.000000   
449   Default   376.000000                            sequin   376.000000   
129   Default   662.000000                           comfort   662.000000   
797   Default   424.000000                         embellhed   424.000000   
333   Default   827.000000                           relaxed   827.000000   
278   Default   680.000000                             lined   680.000000   
188   Default   875.000000                              wrap   875.000000   
212   Default   745.000000                             overd   745.000000   
1336  Default   352.000000                             tulle   352.000000   
171   Default   454.000000                           through   454.000000   
...       ...          ...                               ...          ...   
789    Topic6    81.097076                        when_doubt    81.274406   
717    Topic6    82.987511                             which    83.169617   
700    Topic6    81.634193                  healthier_planet    81.816238   
697    Topic6    81.634193          grow_which_means_happier    81.816238   
713    Topic6    81.634193                             stuff    81.816238   
696    Topic6    81.634193                        grow_which    81.816238   
695    Topic6    81.634193                        good_stuff    81.816238   
706    Topic6    81.634193                   less_pesticides    81.816238   
712    Topic6    81.634193                        pesticides    81.816238   
698    Topic6    81.634193                           happier    81.816238   
699    Topic6    81.634193                         healthier    81.816238   
714    Topic6    81.634193                       supermarket    81.816238   
705    Topic6    81.634193              know_like_good_stuff

In [10]:
model.save('lda.model')

In [11]:
def explore_topic(lda_model, topic_number, topn, output=True):
    terms = []
    for term, frequency in lda_model.show_topic(topic_number, topn=topn):
        terms += [term]
        if output:
            print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))   
    return terms

In [12]:
print(u'{:20} {}'.format(u'term', u'frequency') + u'\n')
for i in range(num_topics):
    print('Topic '+str(i)+' |---------------------\n')
    tmp = explore_topic(model,topic_number=i, topn=10, output=True )

term                 frequency

Topic 0 |---------------------

fully_lined          0.051
fully                0.051
more                 0.043
sequin               0.031
embellhed            0.025
definitely           0.021
more_definitely      0.021
extra                0.020
days                 0.019
call                 0.018
Topic 1 |---------------------

maxi                 0.056
skirt                0.031
close                0.028
slim                 0.027
little               0.022
length               0.020
narrow               0.019
sits_close           0.019
sits                 0.019
slim_narrow          0.019
Topic 2 |---------------------

lining               0.033
floral               0.030
lace                 0.025
fastening            0.025
midi                 0.021
over                 0.019
wrap                 0.018
ruffle               0.018
lined                0.015
true                 0.014
Topic 3 |---------------------

button               0.046
min

### Based on the above, giving a general name of each cluster.

In [13]:
top_labels = {0: 'floral', 1:'drapes', 2:'shirt', 3:'lace skirt', 4:'black bodysuit', 5:'conscious'}

In [14]:
from collections import OrderedDict
def get_doc_topic_dist(model, corpus, kwords=True):
    
    '''
    LDA transformation, for each doc only returns topics with non-zero weight
    This function makes a matrix transformation of docs in the topic space.
    '''
    top_dist =[]
    keys = []

    for d in corpus:
        tmp = {i:0 for i in range(num_topics)}
        tmp.update(dict(model[d]))
        vals = list(OrderedDict(tmp).values())
        top_dist += [array(vals)]
        if kwords:
            keys += [array(vals).argmax()]

    return array(top_dist), keys

In [15]:
top_dist, lda_keys= get_doc_topic_dist(model, corpus, True)


In [16]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(top_dist)

In [17]:
df1['X_tsne'] =X_tsne[:, 0]
df1['Y_tsne'] =X_tsne[:, 1]
df1['clusters'] = lda_keys

/Users/kellyho/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/kellyho/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/kellyho/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [18]:
cluster_colors = {0: 'blue', 1: 'green', 2: 'yellow', 3: 'red', 4: 'skyblue', 5:'salmon', 6:'orange', 7:'maroon', 8:'crimson', 9:'black', 10:'gray'}

df1['colors'] = df1['clusters'].apply(lambda l: cluster_colors[l])

/Users/kellyho/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
df1

Unnamed: 0                                            alltext brand  \
3136         3136  ASOS DESIGN blouse with frill shoulder Lightwe...   NaN   
3137         3137  A Star Is Born Going Out Festival High Neck Bo...   NaN   
3138         3138  Converse Cons Skate Boarding Long Sleeve T-Shi...   NaN   
3139         3139  Vans White Finish Line Heritage Long Sleeve T-...   NaN   
3140         3140  ASOS DESIGN x glaad& Curve t-shirt with high n...   NaN   
3141         3141  ASOS DESIGN Tall t-shirt with textured neon ci...   NaN   
3142         3142  adidas Training All Me Vfa Bra In Black Gym mo...   NaN   
3143         3143  Converse Long Sleeve T-Shirt In Pink Crew neck...   NaN   
3144         3144  ASOS DESIGN Curve bardot shirred top Off-shoul...   NaN   
3145         3145  Calvin Klein Jeans box logo t shirt Wear it ag...   NaN   
3146         3146  ASOS DESIGN Wrap Blouse With Kimono Sleeve In ...   NaN   
3147         3147  ASOS DESIGN oversized denim shirt in midwash b...   NaN   
3148         3148  ASOS DESIGN tank body with lace open back Roun...   NaN   
3149         3149  ASOS DESIGN button front cami with ruffle deta...   NaN   
3150         3150  Parallel Lines plunge halter neck body All-whi...   NaN   
3151         3151  ASOS DESIGN Petite ultimate tank in black One ...   NaN   
3152         3152  Kiss The Sky Tie Back Crop Top In Lace Two-Pie...   NaN   
3153         3153  Wednesday's Girl Crop Top With Tie Front In St...   NaN   
3154         3154  Tommy Hilfiger Color Block Flag Bralette Scoop...   NaN   
3155         3155  Juicy Couture Black Label Velour Hoodie Welcom...   NaN   
3156         3156  Kiss The Sky Floral Print Crop Top Two-Piece C...   NaN   
3157         3157  Neon Rose Kimono In Floral Two-Piece Floral pr...   NaN   
3158         3158  Glamorous Petite High Neck Crop Top With Tie B...   NaN   
3159         3159  South Beach Tie Dye Strappy Bra Top In Green T...   NaN   
3160         3160  Reclaimed Vintage Inspired Tropical Print Band...   NaN   
3161         3161  Pull&Bear Two-piece Stripe Button Detail Front...   NaN   
3162         3162  PrettyLittleThing Multi Stripe Tie Front Beach...   NaN   
3163         3163  Glamorous Tall tie front top two-piece Co-ord ...   NaN   
3164         3164  Native Rose Festival Kimono In Embroidered Lac...   NaN   
3165         3165  Puma Exclusive To Asos Active Long Sleeve Mesh...   NaN   
...           ...                                                ...   ...   
10185       10185  Outrageous Fortune ruffle wrap midi dress with...   NaN   
10186       10186  ASOS DESIGN Scuba One Shoulder Frill Printed M...   NaN   
10187       10187  ASOS DESIGN bonded origami fold shift mini dre...   NaN   
10188       10188  Chi Chi London cap sleeve lace pencil dress So...   NaN   
10189       10189  True Violet cross front bardot skater prom dre...   NaN   
10190       10190  Y.A.S Ditsy Floral Midi Dress With Ruffle Hem ...   NaN   
10191       10191  ASOS DESIGN twist mini dress in chain print Th...   NaN   
10192       10192  ASOS DESIGN premium frill shoulder one sleeve ...   NaN   
10193       10193  ASOS DESIGN Bandeau crop top midi prom dress B...   NaN   
10194       10194  ASOS DESIGN casual popper front mini dress wit...   NaN   
10195       10195  ASOS DESIGN Asymmetric Tux Mini Dress One-shou...   NaN   
10196       10196  Fashion Union plunge front dress in floral Say...   NaN   
10197       10197  PrettyLittleThing Shirt Dress New garms?,We’ve...   NaN   
10198       10198  ASOS DESIGN mono color block t-shirt dress Som...   NaN   
10199       10199  ASOS DESIGN button through mini skater dress i...   NaN   
10200       10200  ASOS DESIGN high neck pleated maxi dress For t...   NaN   
10201       10201  Boohoo stripe midi wrap dress Do you believe i...   NaN   
10202       10202  ASOS DESIGN Petite mini smock dress This is a ...   NaN   
10203       10203  Oasis hanky hem midi dress in black & white st...   NaN   
10204       10204  Missguided

In [20]:
from bokeh.plotting import figure, show, output_notebook, save
from bokeh.models import HoverTool, value, LabelSet, Legend, ColumnDataSource
output_notebook()

Loading BokehJS ...

In [21]:
source = ColumnDataSource(dict(
    x=df1['X_tsne'],
    y=df1['Y_tsne'],
    color=df1['colors'],
    label=df1['clusters'],
    topic_cluster= df1['clusters'],
    title= df1[u'product_name'],
    img_url = df1['img_url'],
    website = df1['website']
))

In [22]:
title = 'T-SNE visualization of topics'

plot_lda = figure(plot_width=1000, plot_height=600,
                     title=title, tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
                     x_axis_type=None, y_axis_type=None, min_border=1)

plot_lda.scatter(x='x', y='y', legend='label', source=source, color='color', alpha=0.8, size=10)

# hover tools
hover = plot_lda.select(dict(type=HoverTool))
hover.tooltips = {"content": "Prod_name: @title, website: @website - Topic: @topic_cluster "}
plot_lda.legend.location = "top_left"

show(plot_lda)

#save the plot
save(plot_lda, '{}.html'.format(title))

/Users/kellyho/anaconda3/lib/python3.6/site-packages/bokeh/io/saving.py:125: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/Users/kellyho/anaconda3/lib/python3.6/site-packages/bokeh/io/saving.py:138: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'/Users/kellyho/Desktop/capstone/Kelly/T-SNE visualization of topics.html'